In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/Users/danieloliveiradebrito/Projetos/fakenews


/Users/danieloliveiradebrito/Projetos/fakenews/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Load dataset

In [66]:
from fakenews.read_data import read_fake_true_br
from sklearn.model_selection import train_test_split
import datasets
import polars as pl

df = read_fake_true_br()

train_full, test = train_test_split(df.to_pandas(),
                              test_size=0.2,
                              random_state=42,
                              shuffle=True,
                              stratify=df["label"]
                            )

train, val = train_test_split(train_full,
                              test_size=0.2,
                              random_state=42,
                              shuffle=True,
                              stratify=train_full["label"]
                            )

ds = datasets.DatasetDict()
ds["train"] = datasets.Dataset.from_pandas(train.reset_index(drop=True))
ds["test"] = datasets.Dataset.from_pandas(test.reset_index(drop=True))
ds["validation"] = datasets.Dataset.from_pandas(val.reset_index(drop=True))


/Users/danieloliveiradebrito/Projetos/fakenews/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# BERT

In [67]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("adalbertojunior/distilbert-portuguese-cased")
tokenizer = AutoTokenizer.from_pretrained("adalbertojunior/distilbert-portuguese-cased", do_lower_case=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at adalbertojunior/distilbert-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [68]:
from fakenews.train import FakeNewsTrainer

faketrue_trainer = FakeNewsTrainer(ds)
tokenized_ds = faketrue_trainer.tokenize_ds()
trainer = faketrue_trainer.get_trainer(tokenized_ds)

/Users/danieloliveiradebrito/Projetos/fakenews/.venv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at adalbertojunior/distilbert-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 573/573 [00:00<00:00, 1548.50 examples/s]


In [69]:
import json

trainer.train()

eval_dict = trainer.evaluate(tokenized_ds["test"])
with open("reports/model_eval/faketrue_br.json", "w") as f:
    json.dump(eval_dict, f)

                                                 
 33%|███▎      | 287/861 [02:57<09:05,  1.05it/s]

{'eval_loss': 0.22044651210308075, 'eval_accuracy': 0.9528795811518325, 'eval_clf_report': {'0': {'precision': 0.9923954372623575, 'recall': 0.9125874125874126, 'f1-score': 0.9508196721311475, 'support': 286.0}, '1': {'precision': 0.9193548387096774, 'recall': 0.9930313588850174, 'f1-score': 0.9547738693467337, 'support': 287.0}, 'accuracy': 0.9528795811518325, 'macro avg': {'precision': 0.9558751379860174, 'recall': 0.952809385736215, 'f1-score': 0.9527967707389406, 'support': 573.0}, 'weighted avg': {'precision': 0.9558114027342263, 'recall': 0.9528795811518325, 'f1-score': 0.9528002211728109, 'support': 573.0}}, 'eval_runtime': 11.7604, 'eval_samples_per_second': 48.723, 'eval_steps_per_second': 6.122, 'epoch': 1.0}


 58%|█████▊    | 500/861 [05:00<03:26,  1.75it/s]

{'loss': 0.2067, 'grad_norm': 6.306834697723389, 'learning_rate': 8.385598141695703e-06, 'epoch': 1.74}


                                                 
 67%|██████▋   | 574/861 [05:53<02:22,  2.01it/s]

...

                                                 
100%|██████████| 861/861 [08:48<00:00,  2.04it/s]

{'eval_loss': 0.14759203791618347, 'eval_accuracy': 0.9703315881326352, 'eval_clf_report': {'0': {'precision': 0.9752650176678446, 'recall': 0.965034965034965, 'f1-score': 0.9701230228471002, 'support': 286.0}, '1': {'precision': 0.9655172413793104, 'recall': 0.975609756097561, 'f1-score': 0.9705372616984402, 'support': 287.0}, 'accuracy': 0.9703315881326352, 'macro avg': {'precision': 0.9703911295235774, 'recall': 0.970322360566263, 'f1-score': 0.9703301422727701, 'support': 573.0}, 'weighted avg': {'precision': 0.9703826236105857, 'recall': 0.9703315881326352, 'f1-score': 0.9703305037377365, 'support': 573.0}}, 'eval_runtime': 11.0167, 'eval_samples_per_second': 52.012, 'eval_steps_per_second': 6.536, 'epoch': 3.0}


100%|██████████| 861/861 [08:50<00:00,  1.62it/s]


{'train_runtime': 530.1289, 'train_samples_per_second': 12.97, 'train_steps_per_second': 1.624, 'train_loss': 0.142498333141225, 'epoch': 3.0}


100%|██████████| 90/90 [00:14<00:00,  6.37it/s]
